In [ ]:
import rospy
import rosbag
import sys
import os

In [ ]:
####################################################
########## Modify the Configuration  Here ##########
####################################################

# Replace with paths for your .bag files
bag_file = "2023-11-03-11-07-11/imus.bag"  
output_bag_file = "2023-11-03-11-07-11/imus_trimmed.bag"

#Start time options (Set only One while other should be None)
# Specify the start time in UNIX time (e.g., 1699024032.0)
start_unix_time = None 
#OR
# Specify the start time offset in seconds from the beginning of the bag 
start_time_offset =  10

# Duration in seconds for the trimmed bag
duration_seconds = 30  

In [ ]:
# Initial Checks. Must pass before you proceed
if not os.path.exists(bag_file):
    raise FileNotFoundError(f"The bag file {bag_file} does not exist.")


if start_unix_time is None and start_time_offset is None:
    raise ValueError(f"You must specify start_unix_time or start_time_offset. Cannot proceed.")

if not start_time_offset is None:
    if start_time_offset < 0:
        raise ValueError(f"Error: Specified start_time_offset {start_time_offset}! must be positive offset. Cannot proceed.")

if duration_seconds < 0:
    raise ValueError(f"Error: Specified duration_seconds {duration_seconds}! must be positive. Cannot proceed.")


In [ ]:
# Open the input bag file
with rosbag.Bag(bag_file, 'r') as input_bag:
    # Get the start time of the bag
    bag_start_time = input_bag.get_start_time()
    bag_end_time = input_bag.get_end_time()

    
    # Determine the start time for trimming
    if start_unix_time:
        start_time = start_unix_time
        if start_unix_time < bag_start_time:
            print(f"Warning: Specified start UNIX time {start_unix_time} is before the bag start time. Using bag start time {bag_start_time} instead.")
            start_time = bag_start_time
        elif start_unix_time > bag_end_time:
            raise ValueError(f"Error: Specified start UNIX time {start_unix_time} is after the bag end time {bag_end_time}. Cannot proceed.")
        else:
            start_time = start_unix_time        
    else:
        start_time = bag_start_time + start_time_offset

    # Determine the end time for trimming
    end_time = start_time + duration_seconds
    if end_time > bag_end_time:
        duration_seconds = bag_end_time - start_time
        print(f"Warning: Specified duration leads to an end time beyond the bag's end time. Adjusting duration to {duration_seconds} seconds.")
        end_time = bag_end_time
    

    print(f'Start trimming bag from {start_time} for a duration of {duration_seconds} seconds')

    last_time = start_time
    with rosbag.Bag(output_bag_file, 'w') as output_bag:
        # Iterate through the messages in the input bag
        for topic, msg, t in input_bag.read_messages(start_time=rospy.rostime.Time.from_sec(start_time), end_time=rospy.rostime.Time.from_sec(end_time)):
            current_time = t.to_sec()
            elapsed_time = current_time - start_time
            progress = (elapsed_time / duration_seconds) * 100

            # Print progress Every second
            if current_time - last_time >= 1: 
                sys.stdout.write(f"\rProcessing bag: {progress:.2f}% complete")
                sys.stdout.flush()
                last_time = current_time
                
            output_bag.write(topic, msg, t)
    
    sys.stdout.write(f"\rProcessing bag: 100.0% complete")
    sys.stdout.flush()
    print()
    print(f"Trimmed bag saved to {output_bag_file}")